In [1]:
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim
from zoneinfo import ZoneInfo
from datetime import datetime
import swisseph as swe
import pandas as pd

In [2]:
path_to_ephe = 'C:\\Users\\Ivan\\ai_astrology\\ephe'
# set path to ephe files
swe.set_ephe_path(path_to_ephe)
# set vedic horo mode
swe.set_sid_mode(swe.SIDM_LAHIRI)

In [1]:
# Проверим, установлена ли система Лахири
#print("Сидерическая система:", swe.get_ayanamsa_name(swe.SIDM_LAHIRI))  # → 'Lahiri'

# Проверим, сколько составляет аянсамша на день рождения
#jd = swe.julday(1987, 1, 20, 10 + 53/60)
#ayanamsa = swe.get_ayanamsa(jd)
#print("Аянсамша (смещение от тропического зодиака):", round(ayanamsa, 4), "°")

In [6]:
ZODIAC_SIGNS = [
    "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева",
    "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"
]

def get_sign_name(degree):
    index = int(degree // 30)
    return ZODIAC_SIGNS[index % 12]

def format_longitude(degree):
    sign_index = int(degree // 30)
    sign_deg = degree % 30
    minutes = (sign_deg - int(sign_deg)) * 60
    return f"{ZODIAC_SIGNS[sign_index]} {int(sign_deg)}° {int(minutes)}′"

In [7]:
def get_coordinates_by_geo(city, country):
    geolocator = Nominatim(user_agent="vedic_chart_app")
    location = geolocator.geocode(f"{city}, {country}")
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError("Местоположение не найдено")

In [8]:
def get_utc_birth_datetime(birth_datetime_str, latitude, longitude):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=latitude, lng=longitude)
    if not tz_name:
        raise ValueError("Не удалось определить часовой пояс для координат")
    tz = ZoneInfo(tz_name)

    dt_local = datetime.strptime(birth_datetime_str, "%d.%m.%Y %H:%M").replace(tzinfo=tz)
    return dt_local.astimezone(ZoneInfo("UTC"))

In [14]:
#def calculate_sidereal_ascendant(jd_ut, latitude, longitude):
#    jd_et = jd_ut + swe.deltat(jd_ut)
#    # 1. Тропический асцендент
#    cusps, ascmc = swe.houses_ex(jd_ut, latitude, longitude, b'C')
#    asc_tropical = ascmc[0]
#    # 2. Аянсамша
#    ayanamsa = swe.get_ayanamsa(jd_et)
#    # 3. Сидерический лагна
#    asc_sidereal = (asc_tropical - ayanamsa) % 360
#    return asc_sidereal

In [18]:
def calculate_vedic_chart(dt_utc, latitude, longitude):
    # Юлианская дата
    hour_decimal = dt_utc.hour + dt_utc.minute / 60 + dt_utc.second / 3600
    jd_ut = swe.julday(dt_utc.year, dt_utc.month, dt_utc.day, hour_decimal)
    ayanamsa = swe.get_ayanamsa(jd)

    # Флаги для сидерических координат
    flags = swe.FLG_SWIEPH | swe.FLG_SIDEREAL

    # Список планет
    planet_ids = [
        swe.SUN, swe.MOON, swe.MERCURY, swe.VENUS, swe.MARS,
        swe.JUPITER, swe.SATURN, swe.URANUS, swe.NEPTUNE, swe.PLUTO,
        swe.MEAN_NODE  # Раху
    ]
    planet_names = [
        "Солнце", "Луна", "Меркурий", "Венера", "Марс",
        "Юпитер", "Сатурн", "Уран", "Нептун", "Плутон", "Раху"
    ]

    result = []

    for i in range(len(planet_ids)):
        planet_id = planet_ids[i]
        planet_name = planet_names[i]

        calc_result = swe.calc(jd_ut, planet_id, flags)
        lon = calc_result[0][0] % 360
        sign = get_sign_name(lon)
        degree_in_sign = lon % 30
        formatted = format_longitude(lon)

        result.append({
            "Планета": planet_name,
            "Долгота (°)": round(lon, 4),
            "Знак": sign,
            "Градус в знаке": round(degree_in_sign, 4),
            "Формат": formatted
        })

    # Кету — противоположная точка Раху
    rahu_lon = result[-1]["Долгота (°)"]
    ketu_lon = (rahu_lon + 180.0) % 360
    ketu_sign = get_sign_name(ketu_lon)
    ketu_deg = ketu_lon % 30
    ketu_formatted = format_longitude(ketu_lon)
    result.append({
        "Планета": "Кету",
        "Долгота (°)": round(ketu_lon, 4),
        "Знак": ketu_sign,
        "Градус в знаке": round(ketu_deg, 4),
        "Формат": ketu_formatted
    })

    # Асцендент — теперь сразу сидерический
    houses = swe.houses_ex(jd_ut, latitude, longitude, b'A')
    asc_sidereal = houses[1][0] % 360 - ayanamsa
    asc_sign = get_sign_name(asc_sidereal)
    asc_deg = asc_sidereal % 30
    asc_formatted = format_longitude(asc_sidereal)
    result.append({
        "Планета": "Асцендент",
        "Долгота (°)": round(asc_sidereal, 4),
        "Знак": asc_sign,
        "Градус в знаке": round(asc_deg, 4),
        "Формат": asc_formatted
    })

    return pd.DataFrame(result)

In [22]:
#birth_datetime_str = "20.01.1987 10:53"
#lat, long = get_coordinates_by_geo('Snezhinsk', 'Russia')
#print(lat, long)
#dt_utc = get_birth_datetime(birth_datetime_str, lat, long)
#print(dt_utc)

In [ ]:
# user data
city = 'Snezhinsk'
country = 'Russia'
birth_date = "20.01.1987"
birth_time = "10:53"
lat, long = 56.05, 60.44

In [12]:
birth_time_str = birth_date + ' ' + birth_time
geo_location = city + 
dt_utc = get_birth_datetime(birth_time_str, lat, long)

1987-01-20 05:53:00+00:00


In [19]:
vedic_chart = calculate_vedic_chart(dt_utc, lat, long)

In [20]:
vedic_chart

,Планета,Долгота (°),Знак,Градус в знаке,Формат
0,Солнце,275.9514,Козерог,5.9514,Козерог 5° 57′
1,Луна,153.5394,Дева,3.5394,Дева 3° 32′
2,Меркурий,280.8926,Козерог,10.8926,Козерог 10° 53′
3,Венера,229.1495,Скорпион,19.1495,Скорпион 19° 8′
4,Марс,344.5226,Рыбы,14.5226,Рыбы 14° 31′
5,Юпитер,327.2349,Водолей,27.2349,Водолей 27° 14′
6,Сатурн,233.7316,Скорпион,23.7316,Скорпион 23° 43′
7,Уран,241.0043,Стрелец,1.0043,Стрелец 1° 0′
8,Нептун,252.7217,Стрелец,12.7217,Стрелец 12° 43′
9,Плутон,196.1476,Весы,16.1476,Весы 16° 8′


In [14]:
D1_TO_D9_MAP = {
    "Овен":      ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Телец":     ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Близнецы":  ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рак":       ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Лев":       ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Дева":      ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Весы":      ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Скорпион":  ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Стрелец":   ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Козерог":   ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Водолей":   ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рыбы":      ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
}

In [15]:
def build_navamsa(df):
    navamsa_signs = []

    for i, row in df.iterrows():
        rashi_sign = row["Знак"]
        degree_in_sign = row["Градус в знаке"]
        planet = row["Планета"]

        if rashi_sign is None or degree_in_sign is None:
            navamsa_signs.append(None)
            continue

        part = int(degree_in_sign // (30 / 9))  # каждая навамша = 3°20′
        d9_sign = D1_TO_D9_MAP[rashi_sign][part]

        # Отладочный вывод для Асцендента
        if planet == "Асцендент":
            print(f"[DEBUG] Асцендент: знак={rashi_sign}, градус={degree_in_sign:.4f}, часть={part}, Навамша={d9_sign}")

        navamsa_signs.append(d9_sign)

    df["Навамша (знак)"] = navamsa_signs
    return df

In [16]:
vedic_chart = build_navamsa(vedic_chart)

[DEBUG] Асцендент: знак=Водолей, градус=29.5202, часть=8, Навамша=Близнецы


In [17]:
vedic_chart

,Планета,Долгота (°),Знак,Градус в знаке,Формат,Навамша (знак)
0,Солнце,275.9514,Козерог,5.9514,Козерог 5° 57′,Водолей
1,Луна,153.5394,Дева,3.5394,Дева 3° 32′,Водолей
2,Меркурий,280.8926,Козерог,10.8926,Козерог 10° 53′,Овен
3,Венера,229.1495,Скорпион,19.1495,Скорпион 19° 8′,Стрелец
4,Марс,344.5226,Рыбы,14.5226,Рыбы 14° 31′,Скорпион
5,Юпитер,327.2349,Водолей,27.2349,Водолей 27° 14′,Близнецы
6,Сатурн,233.7316,Скорпион,23.7316,Скорпион 23° 43′,Водолей
7,Уран,241.0043,Стрелец,1.0043,Стрелец 1° 0′,Овен
8,Нептун,252.7217,Стрелец,12.7217,Стрелец 12° 43′,Рак
9,Плутон,196.1476,Весы,16.1476,Весы 16° 8′,Водолей


In [ ]:
# pyswisseph version used 2.10.3.2 
def get_asc2(year, month, day, hour, minute, second, tzoffset, latitude, longitude):   
    #swe.set_sid_mode(swe.SIDM_LAHIRI, 0, 0)  # Set the Ayanamsa
    #flags = swe.FLG_SWIEPH + swe.FLG_SPEED + swe.FLG_SIDEREAL
    utcc = swe.utc_time_zone(year, month, day, hour, minute, second, tzoffset)
    print(utcc)
    jdet, jdut = swe.utc_to_jd(utcc[0], utcc[1], utcc[2], utcc[3],utcc[4], utcc[5])
    print(jdut)
    swe.set_sid_mode(swe.SIDM_LAHIRI, 0, 0)  # Set the Ayanamsa
    
    #cusps, ascmc = swe.houses_ex(jd_ut, latitude, longitude, b'C', swe.FLG_SWIEPH + swe.FLG_SPEED + swe.FLG_SIDEREAL)
    cusps, ascmc = swe.houses_ex(jd_ut, latitude, longitude, b'C')
    ascendant = ascmc[0]
    sign = int(ascendant/30) + 1
    return sign, ascendant